In [1]:
import pandas as pd 
import numpy as np
from datetime import date, datetime, timedelta
import lxml.html
import json
import requests 
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
url='https://www.vivanuncios.com.mx/s-venta-propiedades-comerciales/distrito-federal/v1c1110l1008p1'
headers={
'Content-Type':'application/x-www-form-urlencoded',
'Origin': 'https://www.vivanuncios.com.mx',
'Referer': 'https://www.vivanuncios.com.mx/',
'sec-ch-ua': '"Chromium";v="104", " Not A;Brand";v="99", "Microsoft Edge";v="104"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': '"macOS"' }
session=requests.Session()
session.verify=False 
r = session.get( url.replace(' ',''), headers=headers )
session.close()
root = lxml.html.fromstring(r.text)
pagina=root.xpath("////*/div[@class = 'additional-attributes-container']")
spot=pagina[0]
pagina_tabla=pd.DataFrame([],columns=['spot','titulo','descripcion','links','imagen'])
pagina_tabla['spot']=spot.xpath("////*/a[@class = 'href-link tile-title-text']/text()")
pagina_tabla['titulo']=root.xpath("////*/div[@class = 'tile-location one-liner']/b/text()")
links=root.xpath("////*/a[@class = 'href-link tile-title-text']/@href")
pagina_tabla['descripcion']=root.xpath("////*/div[@class = 'expanded-description']/text()")
links=['https://www.vivanuncios.com.mx' + i for i in links ]
pagina_tabla['links']=links
pagina_tabla['imagen']=root.xpath("////*/div[@class = 'bolt-image bolt-image-hoverzoom']/picture/img/@src")

In [2]:
def check_inside(c_json,nombre,dev):
  if nombre in c_json:
        return(c_json[nombre] )
  else: 
        return(dev)

def get_detail(url):
  requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
  headers={
  'Content-Type':'application/x-www-form-urlencoded',
  'Origin': 'https://www.vivanuncios.com.mx',
  'Referer': 'https://www.vivanuncios.com.mx/',
  'sec-ch-ua': '"Chromium";v="104", " Not A;Brand";v="99", "Microsoft Edge";v="104"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"' }
  session=requests.Session()
  session.verify=False 
  r = session.get( url.replace(' ',''), headers=headers )
  session.close()
  root = lxml.html.fromstring(r.text)
  c_json=json.loads(root.xpath("////*/script[@type = 'application/ld+json']/text()")[0])
  lat=check_inside(c_json[0]['geo'],'latitude',0)
  lon=check_inside(c_json[0]['geo'],'longitude',0)
  precio=check_inside(c_json[0]['offers'],'price',0)
  moneda=check_inside(c_json[0]['offers'],'priceCurrency',0)
  fec_ini=check_inside(c_json[0]['offers'],'availabilityStarts',0)
  fec_end=check_inside(c_json[0]['offers'], 'availabilityEnds',0)
  features=root.xpath("////*/div[@class = 'category-detail']/div/span[@class = 'pri-props-name']/text()")
  fea_values=root.xpath("////*/div[@class = 'category-detail']/div/span[@class = 'pri-props-value']/text()")
  fea_json={}
  for i in range(0,len(features)):
    fea_json[features[i]]= fea_values[i] 
  return([lat,lon,precio,moneda,fec_ini,fec_end,fea_json])

In [3]:
cols_detail=['lat','lon','price','currency','fech_ini','fec_end','fea_json']
vivanuncios=pagina_tabla.join(pd.DataFrame([get_detail(j) for j in pagina_tabla.links],columns=cols_detail))

In [ ]:
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
headers={
'Content-Type':'application/x-www-form-urlencoded',
'Origin': 'https://www.vivanuncios.com.mx',
'Referer': 'https://www.vivanuncios.com.mx/',
'sec-ch-ua': '"Chromium";v="104", " Not A;Brand";v="99", "Microsoft Edge";v="104"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': '"macOS"' }
session=requests.Session()
session.verify=False 
r = session.get( url.replace(' ',''), headers=headers )
session.close()
root = lxml.html.fromstring(r.text)
r1=root.xpath("//script[contains(.,'(function(){var w=window;w.$components=(')]")[0].text
step_1 = r1.split('var w=window;w.$components=(w.$components||[]).concat({"o":{"r":"M","w":[["s0-0-2-0-1-41-104-3-relatedItemsV2",0,{},')
step_2 = " ".join(step_1[1].split())
step_3 = step_2.split('var')
step_4= step_3[0].replace('}, {','}xxx{').split('xxx')
step_4[0]

In [5]:
vivanuncios

,spot,titulo,descripcion,links,imagen,lat,lon,price,currency,fech_ini,fec_end,fea_json
0,Edificio en Xochimilco,Amp San Marcos Nte,Edificio de 14 departamentos en esquina de 277...,https://www.vivanuncios.com.mx/a-venta-edifici...,https://img.vivanuncios.com.mx/api/v1/mx-pt10-...,19.27096,-99.11355,10636000,MXN,2022-08-26,2022-11-22,"{'Construídos': '800', 'Recámara(s)': '6+', 'G..."
1,ESTRENA OFICINA DE 109M2 BALCON ROOF COMUN VIG...,Cuauhtémoc,ESTRENA OFICINA 109 M2 HABITABLES \r\nENTREGA ...,https://www.vivanuncios.com.mx/a-venta-oficina...,https://img.vivanuncios.com.mx/api/v1/mx-pt10-...,19.43243,-99.17034,4500065,MXN,2022-08-26,2022-11-23,"{'Garage': '1', 'Terreno': '122', 'Vendedor(a)..."
2,PLAZA COMERCIAL EN VENTA (COMPLETA) RECIÉN TER...,San José de los Cedros 2da Secc,"EDIFICIO DE LOCALES COMERCIALES NUEVO, RECIENT...",https://www.vivanuncios.com.mx/a-venta-locales...,https://img.vivanuncios.com.mx/api/v1/mx-pt10-...,0.00000,0.00000,18000000,MXN,2022-08-26,2022-11-23,"{'Garage': '4+', 'Terreno': '370', 'Vendedor(a..."
3,"Héroes del 47, San Diego Churubusco, Coyoacán",San Diego Churubusco,¡Excelente edificio actualmente habilitado com...,https://www.vivanuncios.com.mx/a-venta-edifici...,https://img.vivanuncios.com.mx/api/v1/mx-pt10-...,19.35150,-99.14961,15250000,MXN,2022-08-26,2022-11-14,"{'Construídos': '638', 'Recámara(s)': '6+', 'T..."
4,"Edificio en venta Simón Bolívar, Venustiano Ca...",Simón Bolívar,¡Magnifica propiedad con tres departamentos! \...,https://www.vivanuncios.com.mx/a-venta-edifici...,https://img.vivanuncios.com.mx/api/v1/mx-pt10-...,19.44659,-99.09830,5700000,MXN,2022-08-26,2022-09-21,"{'Construídos': '383', 'Recámara(s)': '6+', 'G..."
5,"OFICINA WTC PISO 35, ideal para consultorio",Nápoles,Gran oportunidad en el centro de negocios más ...,https://www.vivanuncios.com.mx/a-venta-oficina...,https://img.vivanuncios.com.mx/api/v1/mx-pt10-...,19.39437,-99.17347,4500000,MXN,2022-08-26,2022-11-05,"{'Terreno': '58', 'Vendedor(a)': 'Inmobiliaria'}"
6,Venta Edificio en Florida,Florida,"ATENCION INVERSIONISTAS y FAMILIAS GRANDES, va...",https://www.vivanuncios.com.mx/a-venta-edifici...,https://img.vivanuncios.com.mx/api/v1/mx-pt10-...,19.35625,-99.18339,11900000,MXN,2022-08-26,2022-11-15,"{'Construídos': '346', 'Recámara(s)': '6+', 'G..."
7,LOCAL EN VENTA PLAZA TULYEHUALCO,Tulyehualco Canal de Garay,Wiggot: pfZGqc\n\r\n\rInmobipro le ofrece loca...,https://www.vivanuncios.com.mx/a-venta-locales...,https://img.vivanuncios.com.mx/api/v1/mx-pt10-...,19.31669,-99.07382,1630000,MXN,2022-08-26,2022-11-23,"{'Terreno': '28', 'Vendedor(a)': 'Inmobiliaria'}"
8,TRASPASO DE LOCALES EN DIFERENTES PLAZAS COMER...,Revolución IMSS,Inmobipro le ofrece unos excelentes locales co...,https://www.vivanuncios.com.mx/a-venta-traspas...,https://img.vivanuncios.com.mx/api/v1/mx-pt10-...,0.00000,0.00000,1800000,MXN,2022-08-26,2022-10-18,{}
9,Venta de Oficina PB en Col. Del Valle Provide...,Col del Valle Centro,"Oficinas en VENTA ubicadas en Col. Del Valle, ...",https://www.vivanuncios.com.mx/a-venta-oficina...,https://img.vivanuncios.com.mx/api/v1/mx-pt10-...,19.38715,-99.16970,2990000,MXN,2022-08-26,2022-10-02,"{'Terreno': '64', 'Vendedor(a)': 'Inmobiliaria'}"


In [ ]:
vivanuncios.to_csv('vivanuncio.csv')